## Descripción del proyecto
El proyecto simula el las operaciones de un aerpuerto
Se simula como los aviones utilizan los recursos del aeropuerto: las pistas, los puntos de embarque y desembarque, las estaciones de combustible y el combustible que tiene el aeropuerto.


In [1]:
import simpy
import random
import pandas as pd

In [2]:
class Airport:

    def __init__(self, env, num_runway, num_bg, num_ag, level, num_gas):
        self.env = env
        self.runways = simpy.PriorityResource( env, capacity = num_runway )
        self.boarding_gates = simpy.Resource( env, capacity = num_bg )
        self.arrival_gates = simpy.Resource( env, capacity = num_ag )
        self.gasoline_capacity = simpy.Container( env , init=level , capacity=level  )
        self.gas_station = simpy.Resource( env, capacity = num_gas )
        self.fill_gas = env.event()

    def bring_gas( self ):
        print(f'{env.now}: Sin gasolina')
        time_waiting = random.randint( 15,20 )
        yield env.timeout( time_waiting )
        print(f'{env.now}: Petrolero llegando')  
        amount = self.gasoline_capacity.capacity - self.gasoline_capacity.level   
        if amount == 0:
            amount = self.gasoline_capacity.capacity
        yield self.gasoline_capacity.put(amount)
        yield self.env.timeout(2)
        self.fill_gas.succeed()
        self.fill_gas = self.env.event()

In [21]:
class Airplane:

    def __init__(self, env, airport, priority, name, capacity_tank, level ):
        self.env = env
        self.airport = airport
        self.priority = priority
        self.name = name
        self.capacity_tank = capacity_tank
        self.level = level
        env.process( self.check_operations() )

        self.dfE = pd.DataFrame()

    def check_operations( self ):
        if self.priority == 0:
            # El avion aterriza
            land_proc = self.env.process( self.use_runway() )
            yield land_proc
            arrival_proc = self.env.process ( self.passengers_arrival() )
            yield arrival_proc
        else :
            # El avion va a despegar
            if self.level < self.capacity_tank:
                fill_proc = env.process( self.fill_fuel() )
                yield fill_proc
                
            boarding_proc = self.env.process( self.passengers_boarding() )
            yield boarding_proc
            takeoff_proc = self.env.process( self.use_runway() )
            yield takeoff_proc

    def use_runway( self ):
        print(f'{self.env.now}: {self.name} esperando una pista')
        df = pd.DataFrame({
            "tiempo": [ self.env.now ]
            , "avion": [ self.name ]
            , "servicio": [ "pista" ]
            , "evento": [ "esperando" ]
        })

        self.dfE = pd.concat( [self.dfE , df] )
        with self.airport.runways.request(priority=self.priority) as req:
            yield req
            print(f'{self.env.now}: {self.name} usando una pista')
            # ver como saber si el evento es aterrizar o despegar y cambiar el DF
            df = pd.DataFrame({
                "tiempo": [ self.env.now ]
                , "avion": [ self.name ]
                , "servicio": [ "pista" ]
            })

            if self.priority == 0:
                df["evento"] = ["aterrizar"]
            else:
                df["evento"] = ["despegar"]

            self.dfE = pd.concat( [self.dfE , df] )
            runway_time = random.randint(1,3)
            yield self.env.timeout(runway_time)
            df = pd.DataFrame({
                "tiempo": [ self.env.now ]
                , "avion": [ self.name ]
                , "servicio": [ "pista" ]
                , "evento": [ "final" ]
            })

            self.dfE = pd.concat( [self.dfE , df] )
            if self.priority == 0:
                print(f'{self.env.now}: {self.name} aterrizo en {runway_time} min')
            else:
                print(f'{self.env.now}: {self.name} despego {runway_time} min')

    def passengers_boarding( self ):
        print(f'{self.env.now}: {self.name} esperando una puerta de embarque')
        df = pd.DataFrame({
            "tiempo": [ self.env.now ]
            , "avion": [ self.name ]
            , "servicio": [ "embarque" ]
            , "evento": [ "esperando" ]
        })

        self.dfE = pd.concat( [self.dfE , df] )
        with self.airport.boarding_gates.request() as req:
            yield req
            print(f'{self.env.now}: {self.name} usando puerta de embarque')
            df = pd.DataFrame({
                "tiempo": [ self.env.now ]
                , "avion": [ self.name ]
                , "servicio": [ "embarque" ]
                , "evento": [ "inicio" ]
            })

            self.dfE = pd.concat( [self.dfE , df] )
            time_boarding = random.randint(30, 60)
            yield self.env.timeout(time_boarding)
            print(f'{self.env.now}: {self.name} subiendo pasajeros por {time_boarding} min')
            df = pd.DataFrame({
                "tiempo": [ self.env.now ]
                , "avion": [ self.name ]
                , "servicio": [ "embarque" ]
                , "evento": [ "final" ]
            })

            self.dfE = pd.concat( [self.dfE , df] )

    def passengers_arrival( self ):
        print(f'{self.env.now}: {self.name} esperando una puerta de desembarque')
        df = pd.DataFrame({
            "tiempo": [ self.env.now ]
            , "avion": [ self.name ]
            , "servicio": [ "desembarque" ]
            , "evento": [ "esperando" ]
        })

        self.dfE = pd.concat( [self.dfE , df] )
        with self.airport.arrival_gates.request() as req:
            yield req
            print(f'{self.env.now}: {self.name} usando puerta de desembarque')
            df = pd.DataFrame({
                "tiempo": [ self.env.now ]
                , "avion": [ self.name ]
                , "servicio": [ "desembarque" ]
                , "evento": [ "inicio" ]
            })

            self.dfE = pd.concat( [self.dfE , df] )
            time_arrival = random.randint(15, 30)
            yield self.env.timeout(time_arrival)
            print(f'{self.env.now}: {self.name} bajando pasajeros por {time_arrival} min')
            df = pd.DataFrame({
                "tiempo": [ self.env.now ]
                , "avion": [ self.name ]
                , "servicio": [ "desembarque" ]
                , "evento": [ "final" ]
            })

            self.dfE = pd.concat( [self.dfE , df] )

    def fill_fuel( self ):
        need = self.capacity_tank - self.level
        if self.airport.gasoline_capacity.level < need:
            print(f'{self.env.now}: {self.name} esperando combustible')
            df = pd.DataFrame({
                "tiempo": [ self.env.now ]
                , "avion": [ self.name ]
                , "servicio": [ "combustible" ]
                , "evento": [ "esperando combustible" ]
            })

            self.dfE = pd.concat( [self.dfE , df] )
            if not self.airport.fill_gas.triggered:
                self.env.process(self.airport.bring_gas())
                yield self.airport.fill_gas
        
        print(f'{self.env.now}: {self.name} esperando una bomba de combustible')
        df = pd.DataFrame({
            "tiempo": [ self.env.now ]
            , "avion": [ self.name ]
            , "servicio": [ "combustible" ]
            , "evento": [ "esperando en la fila" ]
        })

        self.dfE = pd.concat( [self.dfE , df] )
        with self.airport.gas_station.request() as req:
            yield req
            print(f'{self.env.now}: {self.name} comenzando a llenar combustible')
            df = pd.DataFrame({
                "tiempo": [ self.env.now ]
                , "avion": [ self.name ]
                , "servicio": [ "combustible" ]
                , "evento": [ "inicio" ]
            })

            self.dfE = pd.concat( [self.dfE , df] )
            time_fill = random.randint(30, 90)
            self.airport.gasoline_capacity.get(need)
            yield self.env.timeout(time_fill)
            print(f'{self.env.now}: {self.name} llenó combustible por {time_fill} min')
            df = pd.DataFrame({
                "tiempo": [ self.env.now ]
                , "avion": [ self.name ]
                , "servicio": [ "combustible" ]
                , "evento": [ "final" ]
            })

            self.dfE = pd.concat( [self.dfE , df] )

In [22]:
class Setting:

    def __init__(self, env, num_airplanes, num_pistas, num_doors, num_gas):
        self.env = env
        self.num_airplanes = num_airplanes
        self.airplanes = []
        self.airport = Airport(self.env, num_pistas,num_doors,num_doors, 5000 , num_gas)
        env.process( self.arriving() )

    def arriving(self):
        for i in range( self.num_airplanes ):
            self.airplanes.append( Airplane(
                env,
                self.airport,
                random.randint(0,1),
                f'Plane{i}',
                1800,
                random.randint(10, 1800)
            ))
            yield self.env.timeout( random.randint( 1 , 60 ) )

In [23]:
random.seed(220)

env = simpy.Environment()

setting = Setting( env , num_airplanes=50, num_pistas=2, num_doors=2, num_gas=1)

env.run( until=720 ) ## 12 horas 

0: Plane0 esperando una pista
0: Plane0 usando una pista
2: Plane0 aterrizo en 2 min
2: Plane0 esperando una puerta de desembarque
2: Plane0 usando puerta de desembarque
16: Plane1 esperando una bomba de combustible
16: Plane1 comenzando a llenar combustible
25: Plane0 bajando pasajeros por 23 min
48: Plane2 esperando una bomba de combustible
59: Plane1 llenó combustible por 43 min
59: Plane1 esperando una puerta de embarque
59: Plane2 comenzando a llenar combustible
59: Plane1 usando puerta de embarque
79: Plane3 esperando una bomba de combustible
94: Plane1 subiendo pasajeros por 35 min
94: Plane1 esperando una pista
94: Plane1 usando una pista
97: Plane1 despego 3 min
104: Plane4 esperando una pista
104: Plane4 usando una pista
107: Plane4 aterrizo en 3 min
107: Plane4 esperando una puerta de desembarque
107: Plane4 usando puerta de desembarque
124: Plane4 bajando pasajeros por 17 min
142: Plane5 esperando una pista
142: Plane5 usando una pista
145: Plane5 aterrizo en 3 min
145: Pla

In [24]:
#concatenar los dataFrame de todos los aviones
df_events = pd.DataFrame()

for d in setting.airplanes:
    df_events = pd.concat([df_events, d.dfE])


In [25]:
df_events

,tiempo,avion,servicio,evento
0,0,Plane0,pista,esperando
0,0,Plane0,pista,aterrizar
0,2,Plane0,pista,final
0,2,Plane0,desembarque,esperando
0,2,Plane0,desembarque,inicio
...,...,...,...,...
0,716,Plane21,pista,esperando
0,716,Plane21,pista,aterrizar
0,718,Plane21,pista,final
0,718,Plane21,desembarque,esperando


# Analisis 

¿Cual es el promedio de aviones atendidos durante 12 horas?

In [35]:
new = df_events.loc[]
new = df_events.groupby(
    ["avion"], 
    as_index=False
).agg({
    "tiempo": ["sum"],
    "servicio": ["count"],
    "evento": ["count"]
})

In [36]:
new

,avion,tiempo,servicio,evento
,,sum,count,count
0,Plane0,31,6,6
1,Plane1,588,9,9
2,Plane10,3474,9,9
3,Plane11,4293,10,10
4,Plane12,2357,6,6
5,Plane13,4686,9,9
6,Plane14,5011,9,9
7,Plane15,3195,6,6
8,Plane16,3550,6,6
